<a href="https://colab.research.google.com/github/alex2844/padavan-builder/blob/main/build.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Settings
import os
import re
import requests
import zipfile
from google.colab import files, drive

REPO = 'https://gitlab.com/hadzhioglu/padavan-ng.git' # @param {"type":"string","placeholder":" "}
os.environ['PADAVAN_REPO'] = REPO;

BRANCH = 'master' # @param {"type":"string","placeholder":" "}
os.environ['PADAVAN_BRANCH'] = BRANCH;

COMMIT = 'HEAD' # @param {"type":"string","placeholder":" "}
os.environ['PADAVAN_COMMIT'] = COMMIT;

TOOLCHAIN_URL = 'https://gitlab.com/api/v4/projects/hadzhioglu%2Fpadavan-ng/packages/generic/toolchain/latest/toolchain.tzst' # @param {"type":"string","placeholder":" "}
os.environ['PADAVAN_TOOLCHAIN_URL'] = TOOLCHAIN_URL;

CONFIG = 'build.config' # @param {"type":"string","placeholder":" "}
os.environ['PADAVAN_CONFIG'] = CONFIG;

THEMES_REPO = 'https://gitlab.com/hadzhioglu/padavan-themes.git' # @param ["https://gitlab.com/hadzhioglu/padavan-themes.git", "https://gitlab.com/hadzhioglu/padavan-themes-lite.git"] {allow-input: true}
os.environ['PADAVAN_THEMES_REPO'] = THEMES_REPO;

THEMES_BRANCH = 'main' # @param {"type":"string","placeholder":" "}
os.environ['PADAVAN_THEMES_BRANCH'] = THEMES_BRANCH;

THEMES = '' # @param {"type":"string","placeholder":" "}
os.environ['PADAVAN_THEMES'] = ' '.join([option.strip() for option in THEMES.split(',')]);

ZIP = 'download' # @param ["download", "save"]

if ZIP == 'save' or "drive/MyDrive/" in CONFIG:
  if not os.path.exists('/content/drive'):
    drive.mount('/content/drive');

In [ ]:
# @title Configuration build {"form-width":"150px"}
# @markdown [![Templates](https://img.shields.io/badge/Templates-config-red.svg)](https://gitlab.com/hadzhioglu/padavan-ng/-/tree/master/trunk/configs/templates)
%%writefile build.config


In [ ]:
# @title Custom post_build script
%%writefile post_build.sh
echo '# FROM https://github.com/alex2844/padavan-builder' > "build.conf";
for v in "${!PADAVAN_@}"; do
  echo "${v}=${!v}" >> "build.conf";
done
echo '' > "build.env";
for v in "${!CONFIG_@}" "${!FW_@}"; do
  echo "${v}=${!v}" >> "build.env";
done

In [ ]:
# @title Build firmware
%%shell
export BUILDER_CLEANUP=false;
export BUILDER_RESET=false;
wget -qO- https://raw.githubusercontent.com/alex2844/padavan-builder/main/build.sh | bash;

In [ ]:
# @title Create zip
with open('build.env', 'r') as f:
  for line in f:
    line = line.strip();
    if line and '=' in line:
      key, value = line.split('=', 1);
      os.environ[key] = value;

zip = os.environ['CONFIG_VENDOR'] + '_' + os.environ['FW_NAME'] + '.zip';
with zipfile.ZipFile(zip, 'w') as zipf:
  zipf.write(os.environ['FW_FILE_NAME']);
  zipf.write('build.conf');
  zipf.write('build.config');

In [ ]:
# @title Download
zip = os.environ['CONFIG_VENDOR'] + '_' + os.environ['FW_NAME'] + '.zip';
if ZIP == 'download':
  files.download(zip);
else:
  ! mv "$zip" "/content/drive/MyDrive/";